# **Multi Server MCP Client**

## 1. **Common transport**
### 단일 Session

In [11]:
import pprint
from langchain_mcp_adapters.client import MultiServerMCPClient

async with MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
) as client:
    pprint.pprint(client.get_tools())

[StructuredTool(name='get_weather', description='Get the weather in a given city', args_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x13896b9c0>),
 StructuredTool(name='get_weather_korea', description='Get the weather in a given city', args_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_weather_koreaArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x138d349a0>)]


### 유지 Session

In [12]:
import pprint
from langchain_mcp_adapters.client import MultiServerMCPClient

# 1. Create a client
client =  MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
)

# 2. Initialize a model
await client.__aenter__()

# 3. Load Tools
tools = client.get_tools()
pprint.pprint(tools)

[StructuredTool(name='get_weather', description='Get the weather in a given city', args_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x13896bf60>),
 StructuredTool(name='get_weather_korea', description='Get the weather in a given city', args_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_weather_koreaArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x13896ab60>)]


## 2. SSE 통신
Model Context Protocol을 Server Sent Events를 통해 구현한 프로토콜. HTTP를 통해 원격서비스에 연결할 수 있다.
- MCP 서버에서 먼저 running되고 있어야 한다.

In [13]:
# 단일 세션
import pprint
from langchain_mcp_adapters.client import MultiServerMCPClient

async with MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
) as client:
    pprint.pprint(client.get_tools())

[StructuredTool(name='get_weather', description='Get the weather in a given city', args_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x138d34540>),
 StructuredTool(name='get_weather_korea', description='Get the weather in a given city', args_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_weather_koreaArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x138d349a0>)]


In [ ]:
# 유지 세션
import pprint
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

models = ChatOpenAI(model="gpt-4o")

# 1. Create a client
client =  MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
)

# 2. Initialize a model
await client.__aenter__()

# 3. Load Tools
tools = client.get_tools()

# 4. Define Prompt
prompt = (
    'You are a smart agent.'
    'Use `weather` tool to get the weather in a given city.'
    'Answer in Korean.'
)
# 5. agent
agent = create_react_agent(models, tools, prompt=prompt)

agent_response = await agent.ainvoke({"messages": "What is the weather in Seoul?"})
agent_response

{'messages': [HumanMessage(content='What is the weather in Seoul?', additional_kwargs={}, response_metadata={}, id='57b6449a-d667-49d4-9e7c-d4379894f59b'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8xcCVv8qXJemHVIRE6aYiz6M', 'function': {'arguments': '{"city":"Seoul"}', 'name': 'get_weather_korea'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 99, 'total_tokens': 117, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6dd05565ef', 'id': 'chatcmpl-BGikX80MdETxFwo35v66WzfBc2Plh', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d58e4861-c7ec-41fd-9975-5a40c611a25d-0', tool_calls=[{'name': 'get_weather_korea', 'args': {'city': 'Seoul'}, 'id': 'call_8xcCVv8qXJemHVIRE6aY

Error in sse_reader: peer closed connection without sending complete message body (incomplete chunked read)


## 3. Std I/O 통신
로컬 프로세스에 적합, 통신을 위한 표준 입출력

In [23]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

server_params = StdioServerParameters(
    command="/Users/choejaehun/Desktop/mcp_tutorial/.venv/bin/python",
    args=["mcp_server_local.py"],
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialization
        await session.initialize()
        
        # Get Tools
        tools = await load_mcp_tools(session)
        pprint.pprint(tools)
        
        # Create Agent
        agent = create_react_agent(models, tools, prompt=prompt)
        agent_response = await agent.ainvoke({"messages": "What is the weather in Seoul?"})
        pprint.pprint(agent_response)

[StructuredTool(name='get_weather', description='Get the weather in a given city', args_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x13a9351c0>)]
{'messages': [HumanMessage(content='What is the weather in Seoul?', additional_kwargs={}, response_metadata={}, id='b02a8455-1ea3-49f7-aa86-047259c8e572'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_m3uJPeRL5BneNbwc4IbUUIzs', 'function': {'arguments': '{"city":"Seoul"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 75, 'total_tokens': 91, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_detail